In [1]:
import pandas as pd
from shapely import wkt, geometry
from shapely.wkt import loads, dumps
from shapely.ops import unary_union, polygonize
import numpy as np
from shapely.geometry import Polygon, Point, LineString, MultiPolygon
import math
import json
import ast
import os
import geopandas as gpd
from geopandas.tools import sjoin
from rtree import index
from pydrive2.drive import GoogleDrive
from pydrive2.auth import GoogleAuth

In [2]:
#Replace this with the path to the shapefile
gadm=r"C:\Users\user\Documents\Python\PythonNotebook\updated_uganda_shapefile\updated_uganda_shapefile.shp"


In [ ]:
# Authenticate to get the file ID of the global oevrlap shapefile
#Run this cell only once!!!. Comment out when not in use to reduce the instances of google auth requests which are limited.
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)
filename = 'Shapefile_for_overlap_check.xlsx'
query = f"title contains '{filename}' and trashed = false"
file_list = drive.ListFile({'q': query}).GetList()
# Print file IDs of matching files
for file in file_list:
    print(f"File Name: {file['title']}, File ID: {file['id']},Date: {file['modifiedDate']}")

In [4]:
#Download global overlap file using the latest id and write to CSV
#This too should be done once to save on time.
file_id = '1OUlENlmX5GKZk6KBRZW64R_HGWuihbcQ'  #Replace with the latest file Id from the above process
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('Shapefile.xlsx')

In [5]:
def read_file(file_path):
    """
    Read either Excel (.xlsx, .xls) or CSV (.csv) files
    
    Parameters:
    file_path (str): Path to the file
    
    Returns:
    pandas.DataFrame: The data from the file
    """
    # Get the file extension
    _, file_extension = os.path.splitext(file_path.lower())
    
    try:
        # Read Excel files
        if file_extension in ['.xlsx', '.xls']:
            return pd.read_excel(file_path)
        # Read CSV files
        elif file_extension == '.csv':
            return pd.read_csv(file_path)
        else:
            raise ValueError(f"Unsupported file format: {file_extension}. Please use Excel (.xlsx, .xls) or CSV (.csv) files.")
            
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {file_path}")
    except Exception as e:
        raise Exception(f"Error reading file: {str(e)}")

In [ ]:
# Read input files
print("Reading input files...")
data = read_file(r"C:\Users\user\Documents\Xavier Polygons 2025\January\31st\1738305868978_cce-polygon-drawings_16072_20250131_064428.xlsx")
global_overlap_df = pd.read_excel('Shapefile.xlsx')

# Rename data columns based on how they were configured
df=data.rename(columns={'Database ID':'id','Polygon ID':'polygon_id',"Pula Box ID":"boxes_pula_id","q_farm_polygon_drawing":"Farm_polygon"})


In [ ]:
#Convert WKT column to geometry
global_overlap_df['geometry'] = gpd.GeoSeries.from_wkt(global_overlap_df.geometry)
global_overlap_df = gpd.GeoDataFrame(global_overlap_df, crs="EPSG:4326", geometry="geometry")
global_overlap_df=global_overlap_df.drop('polygon_id', axis=1)
global_overlap_df=global_overlap_df.rename(columns={'db_polygon_id':'polygon_id'})
print(f"Global Overlap df size: {len(global_overlap_df)}")

In [8]:
def literal_return(val):
    """
    Safely evaluate string representation of Python literals
    Returns the original value if evaluation fails
    """
    try:
        return ast.literal_eval(val)  
    except (ValueError, SyntaxError) as e:
        return val


In [ ]:

# First, identify rows with NAs in Farm_polygon
na_rows = df[df['Farm_polygon'].isna()].copy()
na_rows['Polygon Status'] = 'redo'

# Initialize a DataFrame to store all invalid geometries
invalid_rows = pd.DataFrame()

# Process non-NA rows
df_valid = df.dropna(subset=["Farm_polygon"]).copy()

valid_geom_rows = []  # List to store rows with valid geometries
geom = []  # List to store the valid geometries
lent = []  # To track lengths of the polygons

for i, row in enumerate(df_valid.itertuples(), 1):
    poly = literal_return(row.Farm_polygon)
    
    # Ensure poly is a list of dictionaries
    if isinstance(poly, list) and all(isinstance(p, dict) for p in poly):
        lent.append(len(poly))
        rw = []

        # Extract longitude and latitude
        for j in range(len(poly)):
            if 'longitude' in poly[j] and 'latitude' in poly[j]:
                holder = poly[j]['longitude'], poly[j]['latitude']
                rw.append(holder)
            else:
                print(f"Row {i} contains a point without longitude/latitude: {poly[j]}")
                # Add to invalid rows
                invalid_row = df_valid.iloc[row.Index:row.Index+1].copy()
                invalid_row['Polygon Status'] = 'redo'
                invalid_rows = pd.concat([invalid_rows, invalid_row])
                continue

        if len(rw) == 1:  # Invalid polygon with single point
            invalid_row = df_valid.iloc[row.Index:row.Index+1].copy()
            invalid_row['Polygon Status'] = 'redo'
            invalid_rows = pd.concat([invalid_rows, invalid_row])
            continue

        if len(rw) > 2:  # Valid polygon
            rw.append(rw[0])  # Close the polygon
            try:
                rw = geometry.Polygon(rw)
                if rw.is_valid:
                    geom.append(rw)
                    valid_geom_rows.append(row.Index)
                else:
                    invalid_row = df_valid.iloc[row.Index:row.Index+1].copy()
                    invalid_row['Polygon Status'] = 'redo'
                    invalid_rows = pd.concat([invalid_rows, invalid_row])
            except Exception as e:
                print(f"Error creating polygon for row {i}: {e}")
                invalid_row = df_valid.iloc[row.Index:row.Index+1].copy()
                invalid_row['Polygon Status'] = 'redo'
                invalid_rows = pd.concat([invalid_rows, invalid_row])
        else:
            print(f"Row {i} resulted in an invalid polygon (fewer than 3 points).")
            invalid_row = df_valid.iloc[row.Index:row.Index+1].copy()
            invalid_row['Polygon Status'] = 'redo'
            invalid_rows = pd.concat([invalid_rows, invalid_row])
    else:
        print(f"Row {i} has invalid polygon data: {poly}")
        invalid_row = df_valid.iloc[row.Index:row.Index+1].copy()
        invalid_row['Polygon Status'] = 'redo'
        invalid_rows = pd.concat([invalid_rows, invalid_row])

# Combine all invalid rows (NAs and invalid geometries)
all_invalid_rows = pd.concat([na_rows, invalid_rows])

# Rename columns
all_invalid_rows=all_invalid_rows.rename(columns={'id':'Database ID','polygon_id':'Polygon ID',"boxes_pula_id":"Pula Box ID"})

# Save invalid rows to Excel
all_invalid_rows.to_csv('Files_to_redo.csv', index=False)

# Create final DataFrame with only valid rows
df = df_valid.loc[valid_geom_rows].copy()
df['geom'] = geom
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geom)
gdf.drop("geom", axis=1, inplace=True)
print(f"DataFrame size: {len(gdf)}")
print(f"Number of rows to redo: {len(all_invalid_rows)}")

In [ ]:
# import shapely## Add Location columns
intersection_df = gdf.copy()
# intersection_df = df.copy()
intersection_df["centroid"] = intersection_df["geometry"].centroid
# intersection_df["centroid"] = df["validated_polygon_location_centroid_coordinates"].apply(lambda x: shapely.wkt.loads(x))

intersection_df=intersection_df.to_crs('epsg:4326')
admin_shapefile = gpd.read_file(gadm)
admin_shapefile=admin_shapefile.to_crs('epsg:4326')
intersection_df = gpd.GeoDataFrame(intersection_df,crs=admin_shapefile.crs,geometry="centroid")
admin_shapefile.columns=admin_shapefile.columns.str.lower()
df_points_intersection = sjoin(intersection_df,admin_shapefile,how='left',predicate='intersects')
df_points_intersection = df_points_intersection.drop(columns = ['index_right'])
df_intersection=df_points_intersection.rename(columns={'name_1':'District','name_4':'Parish','name_3':'Subcounty','name_2':'County'})
location_cols = ['polygon_id','District','County',"Subcounty",'Parish']
location_details=df_intersection[location_cols]
location_details=location_details.drop_duplicates(subset='polygon_id')
# data2=pd.merge(data,location_details,on=["polygon_id"],how="left")
gdf2=pd.merge(gdf,location_details,on=["polygon_id"],how="left")
print("Data size: ",len(gdf2))


In [ ]:
# Global Overlapping Shapefile
new_df_ids = gdf2['polygon_id']
gdf2 = gdf2.to_crs('epsg:4326')

global_overlap_df= global_overlap_df[~global_overlap_df['polygon_id'].isin(new_df_ids)]
global_overlap_df=global_overlap_df[global_overlap_df['Parish'].isin(gdf2["Parish"])]
print(f"Global overlaping shapefile size: {len(global_overlap_df)}")
#Write the global overlap file to csv
# global_overlap_df.to_csv('Shapefile_for_overlap_check.csv', index=False)
polygon_df2=global_overlap_df[(["polygon_id", "geometry"])]
polygon_df2 = polygon_df2.rename_geometry('geometry_global_overlap_data')
intersection_df = gpd.sjoin(gdf2, global_overlap_df, how='inner', predicate='intersects')
intersection_df = intersection_df.rename_geometry('geometry_data')
intersection_df=pd.merge(intersection_df,polygon_df2,right_on="polygon_id",left_on="polygon_id_right",how="left")
intersection_df=intersection_df.drop('polygon_id', axis=1)
intersection_df=intersection_df.rename(columns={'polygon_id_left':'polygon_id'})
# intersection_df.to_csv(newpath+"Global_overlap_data.csv",index=False)
print(f"Polygons with Global overlaps: {len(intersection_df)}")

In [ ]:
# Remove duplicate polygons
# Count the number of duplicates
duplicate_count = gdf2.duplicated(subset=['polygon_id'], keep='first').sum()

# Print the number of duplicates
print(f"Number of duplicates found: {duplicate_count}")

# Remove duplicates
gdf2['is_duplicate'] = gdf2.duplicated(subset=['polygon_id'], keep='first')
gdf2 = gdf2.drop_duplicates(subset=['polygon_id'], keep='first')


In [ ]:
# Handle triangular polygons
def identify_and_simplify_triangles(gdf2, offset_percentage=0.32, iterations=1):

    # Identifies triangular polygons and applies QGIS-style simplification that converts
    # triangle vertices into sides of a new 6-sided polygon, maintaining shared sides.
    
    def is_triangle(polygon):
        return len(set(polygon.exterior.coords[:-1])) == 3

    def qgis_style_simplify(geometry, offset_percentage):
        if not is_triangle(geometry):
            return geometry
        
        # Get original vertices (excluding closing point)
        vertices = list(geometry.exterior.coords)[:-1]
        
        # Function to get interpolated points along original sides
        def get_side_points(p1, p2, ratio):
            # Get point at ratio distance from p1 to p2
            x = p1[0] + (p2[0] - p1[0]) * ratio
            y = p1[1] + (p2[1] - p1[1]) * ratio
            # Get point at same ratio from p2 to p1
            x2 = p2[0] + (p1[0] - p2[0]) * ratio
            y2 = p2[1] + (p1[1] - p2[1]) * ratio
            return (x, y), (x2, y2)

        # Generate new vertices for 6-sided polygon
        new_vertices = []
        for i in range(3):
            # Get current and next vertex
            p1 = vertices[i]
            p2 = vertices[(i + 1) % 3]
            
            # Get two points along this side
            point1, point2 = get_side_points(p1, p2, offset_percentage)
            new_vertices.extend([point1, point2])

        # Create new polygon
        new_polygon = Polygon(new_vertices)
        
        # Ensure the new polygon is valid
        if not new_polygon.is_valid:
            new_polygon = new_polygon.buffer(0)
        
        return new_polygon

    # Create a copy of the input GeoDataFrame
    result_gdf = gdf2.copy()
    
    # Filter triangles and apply simplification directly
    mask = result_gdf.geometry.apply(is_triangle)
    triangle_count = mask.sum()  # Count the number of triangles
    result_gdf.loc[mask, 'geometry'] = result_gdf[mask].geometry.apply(
        lambda x: qgis_style_simplify(x, offset_percentage)
    )
    
    corrected_count = triangle_count  # All triangles found are corrected
    
    # Print out the number of triangles found and corrected
    print(f"Number of triangles found: {triangle_count}")
    print(f"Number of triangles corrected: {corrected_count}")
    
    return result_gdf

# Process your existing gdf2
gdf2 = identify_and_simplify_triangles(gdf2)

In [ ]:
# Apply tolerance of 2 meters
tolerance=2

# Reproject to a metric CRS (e.g., UTM zone 33N or EPSG:3857)
gdf2 = gdf2.to_crs(epsg=3857)  # Preserves all columns

# Remove spikes from polygons
gdf2["geometry"] = gdf2["geometry"].buffer(-tolerance).buffer(tolerance)

# Remove Duplicate vertices with tolerance
gdf2["geometry"] = gdf2.remove_repeated_points(tolerance).geometry

# Convert back to WGS84
gdf2 = gdf2.to_crs(epsg=4326)  # All columns are still retained

In [ ]:
# Convert geometry column to WKT strings
global_overlap_df['geometry'] = global_overlap_df['geometry'].apply(lambda x: x.wkt)
gdf2['geometry'] = gdf2['geometry'].apply(lambda x: x.wkt)

In [ ]:
# Remove duplicate vertices
def remove_duplicate_vertices(geometry_str):
    polygon = loads(geometry_str)
    coords = list(polygon.exterior.coords)
    # Remove duplicate consecutive points
    unique_coords = []
    for i in range(len(coords)):
        if i == 0 or coords[i] != coords[i-1]:
            unique_coords.append(coords[i])
    # Ensure the polygon is closed
    if unique_coords[0] != unique_coords[-1]:
        unique_coords.append(unique_coords[0])
    return Polygon(unique_coords)

gdf2['geometry'] = gdf2['geometry'].apply(lambda x: dumps(remove_duplicate_vertices(x)))

In [15]:
# Function for calculation of the area in Acres
def calculate_area_acres(geometry_str):
    polygon = loads(geometry_str)
    return polygon.area * 111319.9 * 111319.9 * np.cos(np.radians(polygon.centroid.y)) * 0.000247105

In [16]:
# Calculate areas and identify small polygons
gdf2['area_acres'] = gdf2['geometry'].apply(calculate_area_acres)
gdf2['is_small_polygon'] = gdf2['area_acres'] < 0.125

In [17]:
# Function to identify self intersections
def is_self_intersecting(geometry_str):
    polygon = loads(geometry_str)
    return not polygon.is_simple

# Check for self-intersections
gdf2['is_self_intersecting'] = gdf2['geometry'].apply(is_self_intersecting)

In [ ]:
# Fix self-intersecting polygons
def fix_self_intersection(geometry_str):
    polygon = loads(geometry_str)
    if not polygon.is_simple:
        boundary = LineString(polygon.exterior.coords)
        valid_polygons = list(polygonize([boundary]))
        
        if valid_polygons:
            largest_polygon = max(valid_polygons, key=lambda p: p.area)
            return largest_polygon
        else:
            return polygon.convex_hull
    return polygon

print("Fixing self-intersecting polygons...")
gdf2['geometry'] = gdf2.apply(
    lambda row: dumps(fix_self_intersection(row['geometry'])) if row['is_self_intersecting'] else row['geometry'],
    axis=1
)

In [ ]:
# Create reference and input polygon lists
print("Creating polygon lists and spatial indices...")
reference_polygons = [loads(geom) for geom in global_overlap_df['geometry']]
input_polygons = [loads(geom) for geom in gdf2['geometry']]

In [20]:
# Create spatial indices
def create_spatial_index(polygons):
    idx = index.Index()
    for pos, poly in enumerate(polygons):
        idx.insert(pos, poly.bounds)
    return idx

reference_idx = create_spatial_index(reference_polygons)
input_idx = create_spatial_index(input_polygons)

In [21]:
# Utility Functions
# 1. Function to check for intersection
def check_intersection_with_index(polygon, spatial_index, all_polygons):
    bounds = polygon.bounds
    potential_matches_idx = list(spatial_index.intersection(bounds))
    return any(polygon.intersects(all_polygons[idx]) for idx in potential_matches_idx)

In [22]:
# 2. Function to rotate polygon
def rotate_polygon(polygon, angle_degrees, origin=None):
    """Rotate polygon by angle (in degrees) around origin (defaults to centroid)."""
    if origin is None:
        origin = polygon.centroid
    
    angle_radians = math.radians(angle_degrees)
    cos_angle = math.cos(angle_radians)
    sin_angle = math.sin(angle_radians)
    
    coords = list(polygon.exterior.coords)
    rotated_coords = []
    
    for x, y in coords:
        # Translate to origin
        dx = x - origin.x
        dy = y - origin.y
        
        # Rotate
        rotated_x = dx * cos_angle - dy * sin_angle
        rotated_y = dx * sin_angle + dy * cos_angle
        
        # Translate back
        final_x = rotated_x + origin.x
        final_y = rotated_y + origin.y
        
        rotated_coords.append((final_x, final_y))
    
    return Polygon(rotated_coords)

In [23]:
# 3. Function to find non-intersecting position
def find_non_intersecting_position(polygon, reference_idx, reference_polygons, input_idx, input_polygons, current_idx):
    """Find a non-intersecting position using both translation and rotation."""
    max_distance = 0.002  # Maximum translation distance
    distance_steps = 8    # Number of steps for translation
    angle_steps = 36      # Number of angles to try for rotation
    max_angle = 360      # Maximum rotation angle
    
    # Original centroid for reference
    original_centroid = polygon.centroid
    
    # Directions for translation
    directions = [(math.cos(2*math.pi*i/8), math.sin(2*math.pi*i/8)) for i in range(8)]
    
    # Try different combinations of translation and rotation
    for distance in np.linspace(0, max_distance, distance_steps):
        for dx, dy in directions:
            # Calculate translation vector
            move_vector = (dx * distance, dy * distance)
            
            # Try different rotation angles
            for angle in np.linspace(0, max_angle, angle_steps):
                # First translate
                translated_coords = [(x + move_vector[0], y + move_vector[1]) 
                                  for x, y in polygon.exterior.coords]
                translated_polygon = Polygon(translated_coords)
                
                # Then rotate
                rotated_polygon = rotate_polygon(translated_polygon, angle)
                
                # Check if this position works
                if not (check_intersection_with_index(rotated_polygon, reference_idx, reference_polygons) or
                       any(rotated_polygon.intersects(other) for i, other in enumerate(input_polygons) 
                           if i != current_idx)):
                    return rotated_polygon, True
    
    # If no valid position found, try more aggressive adjustments
    for distance in np.linspace(max_distance, max_distance * 2, distance_steps):
        for dx, dy in directions:
            move_vector = (dx * distance, dy * distance)
            for angle in np.linspace(0, max_angle, angle_steps):
                translated_coords = [(x + move_vector[0], y + move_vector[1]) 
                                  for x, y in polygon.exterior.coords]
                translated_polygon = Polygon(translated_coords)
                rotated_polygon = rotate_polygon(translated_polygon, angle)
                
                if not (check_intersection_with_index(rotated_polygon, reference_idx, reference_polygons) or
                       any(rotated_polygon.intersects(other) for i, other in enumerate(input_polygons) 
                           if i != current_idx)):
                    return rotated_polygon, True
    
    return polygon, False


In [24]:

def check_any_intersection(polygon, reference_polygons, input_polygons, current_idx):
    """Check if polygon intersects with any reference or input polygon (except itself)."""
    # Check against reference polygons
    if any(polygon.intersects(ref_poly) for ref_poly in reference_polygons):
        return True
    
    # Check against other input polygons
    return any(polygon.intersects(other) for i, other in enumerate(input_polygons) 
              if i != current_idx)


In [ ]:
# Process polygons in batches
print("Processing polygons...")
batch_size = 1000
total_rows = len(gdf2)
intersection_count = 0
fixed_count = 0
unfixed_indices = []
scaling_count = 0

for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)
    batch_indices = gdf2.index[start_idx:end_idx]
    
    for idx in batch_indices:
        row_position = gdf2.index.get_loc(idx)
        current_polygon = loads(gdf2.at[idx, 'geometry'])
        
        # Scale up if too small
        if gdf2.at[idx, 'is_small_polygon']:
            scaling_count += 1
            current_area = gdf2.at[idx, 'area_acres']
            target_area = 0.250000000000  # Target area in acres
            scale_factor = math.sqrt(target_area / current_area)
            
            centroid = current_polygon.centroid
            scaled_coords = [(centroid.x + (x - centroid.x) * scale_factor,
                            centroid.y + (y - centroid.y) * scale_factor)
                           for x, y in current_polygon.exterior.coords]
            current_polygon = Polygon(scaled_coords)
            
            # Update the area after scaling
            gdf2.at[idx, 'geometry'] = dumps(current_polygon)
            gdf2.at[idx, 'area_acres'] = calculate_area_acres(dumps(current_polygon))
            gdf2.at[idx, 'is_small_polygon'] = False
        
        # Check and fix intersections
        if check_any_intersection(current_polygon, reference_polygons, input_polygons, row_position):
            intersection_count += 1
            original_polygon = current_polygon
            current_polygon, was_fixed = find_non_intersecting_position(
                current_polygon,
                reference_idx,
                reference_polygons,
                input_idx,
                input_polygons,
                row_position
            )
            if was_fixed:
                fixed_count += 1
            else:
                unfixed_indices.append(idx)
        
        # Update polygon in dataframe and input_polygons list
        gdf2.at[idx, 'geometry'] = dumps(current_polygon)
        input_polygons[row_position] = current_polygon
        
        # Update input spatial index
        input_idx.delete(row_position, input_polygons[row_position].bounds)
        input_idx.insert(row_position, current_polygon.bounds)
    
    print(f"Processed {end_idx}/{total_rows} polygons. Found {intersection_count} intersections, "
          f"Fixed {fixed_count}, Unfixed {len(unfixed_indices)}, Scaled up {scaling_count}")

# Create separate dataframes for fixed and unfixed cases
unfixed_gdf = gdf2.loc[unfixed_indices].copy()
fixed_gdf = gdf2.drop(unfixed_indices)

In [ ]:
# Convert back to coordinate format
def convert_wkt_to_coordinates(wkt_str):
    polygon = loads(wkt_str)
    coords = list(polygon.exterior.coords)[:-1]
    return json.dumps([{
        'latitude': y,
        'longitude': x
    } for x, y in coords])

print("Converting processed polygons back to coordinate format...")
fixed_gdf['Farm_polygon'] = fixed_gdf['geometry'].apply(convert_wkt_to_coordinates)


In [ ]:
# Final analysis
"""
fixed_gdf['area_acres'] = fixed_gdf['geometry'].apply(calculate_area_acres)
fixed_gdf['is_small_polygon'] = fixed_gdf['area_acres'] < 0.125
fixed_gdf['is_self_intersecting'] = fixed_gdf['geometry'].apply(is_self_intersecting)
"""

In [ ]:
# Print results
print(f"\nProcessing Results:")
print("-----------------")
print(f"1. Duplicates removed: {fixed_gdf['is_duplicate'].sum()} polygons")
print(f"2. Self-intersecting polygons remaining: {fixed_gdf['is_self_intersecting'].sum()}")
print(f"3. Small polygons remaining: {fixed_gdf['is_small_polygon'].sum()}")
print(f"4. Total intersections found: {intersection_count}")
print(f"5. Successfully fixed intersections: {fixed_count}")
print(f"6. Unable to fix: {len(unfixed_indices)}")

In [ ]:
# Save results
print("Saving results...")

# Save unfixed cases to CSV
unfixed_gdf.to_csv('Unfixed_intersections.csv', index=True)
print(f"\nUnfixed cases saved to 'unfixed_intersections.csv'")

fixed_gdf=fixed_gdf.rename(columns={'id':'Database ID','polygon_id':'Polygon ID',"boxes_pula_id":"Pula Box ID"})
columns_to_drop = ['geometry', 'area_acres', 'is_small_polygon', 'is_self_intersecting', 
                  'is_duplicate', 'intersects_others', 'District', 'County', 'Subcounty', 'Parish']
final_df = fixed_gdf.drop(columns=columns_to_drop, errors='ignore')
final_df['Assignee'] = 'Xavier'
final_df.to_csv("Cleaned_polygon_drawings.csv", index=False)
print("\nProcessing complete. Results saved to: cleaned-polygon-drawings.csv")